**Exercício 1: Análise de Internações no Sistema de Saúde Brasileiro**

<p>Você foi contratado(a) para fazer uma análise apurada do número de internações no
sistema de saúde brasileiro. Esta análise é de extrema importância para tomada de
decisões que deverão contribuir para melhorias no sistema e planejamento estratégico.
Os dados em anexo (case_internacao_SUS.xls) são referentes às internações que
ocorreram no país durante o período de dezembro de 2017 a julho de 2019, separados
por região e unidade de federação (Fonte: Ministério da Saúde - Sistema de Informações
Hospitalares do SUS (SIH/SUS)).
Acesse http://tabnet.datasus.gov.br/cgi/sih/sxdescr.htm para obter uma descrição
detalhada dos dados.</p>

Desafios:<br>
**1. Tratamento dos dados**

    1.1. Muitas vezes, cerca de 70% do tempo de um projeto é despendido na coleta e tratamento dos dados. Sabendo disso, leia o arquivo e o transforme de modo a ter mais facilidade em analisar os dados. Lembre-se que essa etapa poderá te dar bons insumos. Portanto, capriche!
 
**2. Análise**

     2.1. Dados tratados, bora explorá-los? Faça uma boa EDA e não esqueça de anotar todos os insights que você obter. Gráficos e informações sem uma boa interpretação não valem, ok?
     
**3. Modelagem**

     3.1. Agora que já tem certa intimidade com os dados, cite pelo menos 2 métodos possíveis para estimar os dados para os meses faltantes. Tente não se complicaraqui. Utilize os métodos mais simples e mais funcionais possíveis. Neste tópico,é importante que argumente o porquê dos métodos recomendados.
            
    3.2. Escolha um desses métodos e estime a) o número de Internações e b) o Valor Total das internações nos períodos faltantes.
    
    3.3. Crie um modelo que preveja a) as Internações, b) o número de Óbitos e c) o Valor Médio de AIH pelos próximos 6 meses. Explique a escolha do modelo e quais parâmetros utilizou para serem input no modelo.
    
**4. Planejamento estratégico**

    4.1. Com base nos dados e nas suas análises, que tipo de estratégia você sugeriria para diminuir o número de internações em hospitais do SUS? E para o Estado de São Paulo? Quais especificidades deveriam ser levadas em conta? 
    
 <p>Ao final, escreva um relatório contendo todos os gráficos gerados bem como as argumentações, análises e insights de sua EDA (item 2.1). Neste relatório também
deverá conter a argumentação do porquê dos métodos citados no item 3.1 são cabíveis
aos dados disponibilizados. Além disso, devem ser incorporadas ao relatório as análises
ou argumentos que te levaram a optar pelo método escolhido para a modelagem
requerida nos itens 3.2 e 3.3. Não esqueça de anexar ao relatório toda argumentação e
apontamentos para o questionamento feito no item 4.1.</p>


# 0.0 Imports

In [1]:
import warnings

import pandas as pd
import inflection
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from IPython.core.display  import HTML
from IPython.display       import Image
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import auto_arima

ModuleNotFoundError: No module named 'inflection'

## 0.1 Funções auxiliares

In [ ]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
jupyter_settings()


pd.set_option('display.float_format', lambda x: '%.3f' % x)


## 0.2 Obtenção dos dados

In [ ]:
data = pd.read_excel('../data/case_internacao_SUS.xls', sheet_name=None)
df_raw = pd.DataFrame()
for key in data.keys():
    df = data[key]
    df['data'] = key
    df_raw = pd.concat([df_raw, df], axis=0)
    

df_raw

# 1.0 Descrição dos dados

## 1.1. Renomeação das colunas

Região/unidade de federação
Internações (AIH aprovadas no período sem considerar prorrogação)
AIH aprovadas (AIH são as autorizações para internação hospitalar - contempla novas e prorrogações)
Valor total -> referente as AIH aprovadas no período, valor aprovado de produção e não necessariamente repassado
Valor serviços hospitalares
Valores serviços hospitalares complemento federal
Valores serviços hospitalares complemento gestor (estadual ou municiapl

In [ ]:
df1 = df_raw.copy()

In [ ]:
cols_old = df1.columns
lower = lambda x: x.lower()

cols_new = list( map( lower, cols_old ) )

df1.columns = cols_new

In [ ]:
df1.columns = ['uf', 'internacoes', 'aih_aprovadas',
       'valor_total', 'valor_servicos_hospitalares',
       'val_serv_hosp_compl_federal', 'val_serv_hosp_compl_gestor',
       'valor_serviços_profissionais', 'val_serv_prof_compl_federal',
       'val_serv_prof_compl_gestor', 'valor_medio_aih', 'valor_medio_intern',
       'dias_permanência', 'media_permanencia', 'obitos', 'taxa_mortalidade',
       'data']

In [ ]:
df1.head()

## 1.2 Separação de dados de estados e região

In [ ]:
#checando quais são os estados e regiões únicos nos dados
df1['uf'].unique()

In [ ]:
#checando as linhas com dados de região vazios
df1[df1['uf'].isnull()]

In [ ]:
# como todas as linhas de uf vazios são vazias, vamos filtrar apenas os dados de uf não nulos
df1 = df1[df1['uf'].notnull()]

In [ ]:
df1.head()

In [ ]:
# estados possuem pontos (.) antes dos seus nomes, portanto vamos filtrar apenas os que são estados
df1 = df1[df1['uf'].str.contains('.', regex=False)]

In [ ]:
df1['uf'].unique()

## 1.3 Limpeza da coluna uf

In [ ]:
df1['uf'] = df1['uf'].apply(lambda x: x.replace('.',''))
df1['uf'] = df1['uf'].apply(lambda x: x.lstrip())
df1['uf'] = df1['uf'].apply(lambda x: x.rstrip())
    
    
    
    
#df1['uf'] = df1['uf'].apply(lambda x: inflection.underscore( x ))

In [ ]:
estados = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}

estados_1 = {v: k for k, v in estados.items()}

df1['uf'] = df1['uf'].map(estados_1)

In [ ]:
df1.head()

## 1.4 Substituição do - por NaN

In [ ]:
for i in df1.columns:
    df1[df1[i] == '-'] = df1[df1[i] == '-'].apply(lambda x: x.replace('-', np.NaN))

## 1.5 Dimensão dos dados

In [ ]:
print('numero_de_linhas: {}'.format(df1.shape[0]))
print('numero_de_colunas: {}'.format(df1.shape[1]))

## 1.6 Tipos dos dados

In [ ]:
df1.dtypes

## 1.7 Check de dados faltantes

In [ ]:
df1.isna().sum()

In [ ]:
df1 = df1[df1['internacoes'].notnull()]

In [ ]:
df1.isna().sum()

## 1.8 Alteração dos tipos dos dados

In [ ]:
#separar mês de ano
df1['mes'] = df1['data'].apply(lambda x: x[0:3])
df1['ano'] = df1['data'].apply(lambda x: x[-2:])

#substituir meses extensos por números
meses = {'jan':'1', 'fev':'2', 'mar':'3', 'abr':'4', 'mai':'5', 'jun':'6', 'jul':'7', 'ago':'8', 'set':'9', 'out':'10', 'nov':'11', 'dez':'12'}
for k,v in meses.items():
    df1['mes'] = df1['mes'].apply(lambda x: x.replace(k,v))

#transformar ano em 04 dígitos
df1['ano'] = df1['ano'].apply(lambda x: '20'+x)

#unir as datas
df1['data'] = df1['ano']+'-'+df1['mes']



In [ ]:
#converter para datetime

df1['data'] = pd.to_datetime(df1['data'], format='%Y-%m')

df1.head()

## 1.9 Preenchimento de dados faltantes 

In [ ]:
# se não houve complemento do governo federal ou do gestor, então os dados faltantes podem ser preenchidos com 0
df1.fillna(0, inplace=True)

In [ ]:
df1.head()

## 1.10 Inserção dos períodos faltantes

In [ ]:
sorted(df1['data'].unique())

Meses faltantes: <br>
2018: jan, fev, jun, out <br>
2019: mar, mai,

In [ ]:
# criar dataset com os timestamps e demais dados vazios

aux = pd.DataFrame(columns=df1.columns)
missing_data = [datetime.date(2018, 1, 1), datetime.date(2018, 2, 1), datetime.date(2018, 6, 1), datetime.date(2018, 10, 1), datetime.date(2019, 3, 1), datetime.date(2019, 5, 1) ]
for i in range(0, len(df1['uf'].unique())):
    for  j in range(0, len(missing_data)):
        aux1 = pd.DataFrame([[df1['uf'].unique()[i],np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN,
                       missing_data[j], np.NaN, np.NaN]], columns=df1.columns)

        aux = pd.concat([aux, aux1])



In [ ]:
aux.head()

In [ ]:
aux['mes'] = pd.DatetimeIndex(aux['data']).month
aux['ano'] = pd.DatetimeIndex(aux['data']).year

In [ ]:
aux['mes'] = aux['mes'].apply(lambda x: str(x))
aux['ano'] = aux['ano'].apply(lambda x: str(x))

In [ ]:
aux.dtypes

In [ ]:
df1.shape

In [ ]:
df1 = pd.concat([df1, aux])

In [ ]:
df1 = df1.sort_values(by='data')

In [ ]:
df1 = df1.reset_index(drop=True)

In [ ]:
df1.head()

In [ ]:
df1['data'] = pd.to_datetime(df1['data'], format='%Y-%m')

In [ ]:
# interpolação, substituição pela média ou mediana, média móvel, autoregressão

In [ ]:
#df1['media_movel_internacoes'] = df1.loc[:,'internacoes'].rolling(window=1).mean()

#df['pandas_SMA_3'] = df.iloc[:,1].rolling(window=3).mean()

In [ ]:
#df1.head()

## 1.12 Estatística descritiva

### 1.12.1 Excluindo período com dados faltantes

In [ ]:
df1_aux = df1[df1['internacoes'].notnull()]

In [ ]:
df1_aux.describe().T

In [ ]:
df1_aux.head()

In [ ]:
aux = df1_aux[['data', 'internacoes']].groupby('data').sum()
aux.describe().T


In [ ]:
aux = df1_aux[['data', 'internacoes']].groupby('data').mean()

In [ ]:
plt.plot(aux)

# 2.0 Feature engineering

In [ ]:
df2 = df1.copy()

In [ ]:
Image( '../references/mindmap.png' )

**Alocação de recursos** <br>
**H1.** O valor total pago por internações deve ser maior na região Sudeste, sendo o estado de São Paulo o estao que deve destinar o maior valor de orçamento para internações. <br>
**H2.** A maior proporção dos complementos federais em relação ao valor total pago devem é a da região nordeste. <br>
**H3.** A maior pproporção dos complementos gestor em relação ao valor total pago deve estar na região sul. <br>
**H4.** O valor médio dos serviços profissionais deve ser maior na região Sudeste e Sul, respectivamente. <br>
**H5.** O maior número de leitos esperado deve estar na região Sudeste. <br> 
**H6.** A região sudeste é a que deve possuir mais leitos (esperados) por habitante <br> 
**H7.** Os estados da região Norte devem ser os que possuem menos leitos (esperados) por habitantes. <br>
**H8.** O valor médio das internações pagas deve ser maior no estado de São Paulo e Distrito Federal, respectivamente <br>
<br>


**Métricas** <br>
**H9.** O tempo médio de permanência deve ser maior em estados do Nordeste <br>
**H10.** A taxa de mortalidade deve ser maior em estados do Norte e Nordesete, respectivamente. <br>
**H11.** A região Sudeste deve concentrar o maior número de prorrogaões de internação que os outros estados.<br>
<br>

**Tempo**<br>
**H12.** O número médio de internações deve ser maior em meses de inverno para todos os estados do Brasil.<br>
**H13.** O tempo médio de internação deve ser maior nos meses de inverno.<br>
**H14.** O comportamento das internações no tempo deve ser igual para ttodos os estados do Brasil<br>

In [ ]:
#região
norte = ['AM', 'PA', 'AP', 'AC', 'RR', 'RO', 'TO']
nordeste = ['MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'SE', 'BA', 'AL']
centro_oeste = ['MT', 'MS', 'GO', 'DF']
sudeste = ['MG', 'ES', 'RJ', 'SP']
sul = ['PR', 'SC', 'RS']

df2['regiao'] = df2['uf'].apply(lambda x: 'norte' if x in norte else 'nordeste' if x in nordeste else 'centro-oeste' if x in centro_oeste else 'sudeste' if x in sudeste else 'sul' if x in sul else np.NaN)


#numero de leitos esperado
df2['no_leitos_ocupado'] = (df2['aih_aprovadas'] * df2['media_permanencia']) / 30



#população de cada estado

populacao = pd.read_csv('../data/tabela6579.csv')

populacao.columns = ['nivel', 'uf', '2017', '2018', '2019']

populacao['uf'] = populacao['uf'].map(estados_1)


df2 = df2.merge(populacao, how='inner', on='uf')

df2.loc[df2['ano'] == '2017', 'populacao'] =df2.loc[df2['ano'] == '2017', '2017']
df2.loc[df2['ano'] == '2018', 'populacao'] =df2.loc[df2['ano'] == '2018', '2018']
df2.loc[df2['ano'] == '2019', 'populacao'] =df2.loc[df2['ano'] == '2019', '2019']
df2 = df2.drop(columns=['nivel', '2017', '2018', '2019'])



#prorrogações

df2['prorrogacoes'] = df2['aih_aprovadas'] - df2['internacoes']

#complemento federal
df2['complemento_federal'] = df2['val_serv_hosp_compl_federal'] + df2['val_serv_prof_compl_federal']

#complemento gestor
df2['complemento_gestor'] = df2['val_serv_hosp_compl_gestor'] + df2['val_serv_prof_compl_gestor']

In [ ]:
df2.head()

# 3.0 Filtragem do dataset

In [ ]:
df3 = df2[['data', 'mes', 'ano', 'uf', 'regiao','internacoes', 'prorrogacoes', 'aih_aprovadas', 'valor_total', 'complemento_federal', 'complemento_gestor', 
           'valor_serviços_profissionais','valor_medio_intern', 'valor_medio_aih', 'no_leitos_ocupado', 'obitos','taxa_mortalidade', 'populacao', 'media_permanencia']]

In [ ]:
#df3 = df3.dropna(axis=0)

In [ ]:
df3.head()

# 4.0 Análise exploraoria de dados (EDA)

In [ ]:
df4 = df3.copy()

## 4.1 Análise univariada

In [ ]:
# análise univariada - distribuição
df4.hist(bins = 25);

## 4.2. Análise bivariada

In [ ]:
# análise bivariada (teste das hipóteses)

### H1. O valor total pago por internações deve ser maior na região Sudeste, sendo o estado de São Paulo o estado que deve destinar o maior valor de orçamento para internações.
**VERDADEIRA.** O valor total pago por internações é maior na região Sudeste, sendo que São Paulo deve receber pelo menos 53.27% dos valor orçamentário para pagamentos de internações da região e 22,724% do valor orçamentário do para este fim do país

In [ ]:
#gasto total mensal por cada região (plot de linhas)
#gasto total mensal por estado

In [2]:
plt.subplot(2, 2, 1)
aux1 = df4[['data', 'regiao', 'valor_total']].groupby(['data','regiao']).sum().reset_index()
sns.lineplot(x='data', y='valor_total', hue='regiao', data=aux1)
plt.title('Valor total destinado para pagamento de internações')
plt.ylabel('Valor pago em internações x 100 milhões de R$')


plt.subplot(2, 2, (2,4))
aux2 = df4[['ano', 'uf', 'valor_total']].groupby('uf').sum().sort_values(by='valor_total', ascending=False).reset_index()
sns.barplot(x='valor_total', y='uf', data=aux2)
plt.title('Valor total em pagamentos de internações do SUS por UF no ano de 2018')
plt.xlabel('Valor total de pagamentos em internações em bilhões de R$');


plt.subplot(2, 2, 3)
aux3 = df4[['data', 'uf', 'valor_total']].groupby(['data','uf']).sum().reset_index()
aux3 = aux3[aux3['uf'].isin(['SP', 'MG', 'PR', 'RS'])]
sns.lineplot(x='data', y='valor_total', hue='uf', data=aux3)
plt.title('Valor pago em internações mensalmente pelos 3 maiores UFs em valor pago ')
plt.ylabel('Valor pago em internações x 100 milhões de R$')

NameError: name 'plt' is not defined

In [ ]:
aux4 = df4[df4['ano']=='2018'][['uf', 'regiao', 'valor_total']].groupby(['regiao', 'uf']).sum().reset_index()

proporcao_valor_internacoes_SP_Brasil = aux4[aux4['uf']== 'SP']['valor_total'].sum()/aux4['valor_total'].sum() *100
print('{}% do valor total repassado para pagamento das internações em 2018 foi para o estado de São Paulo'.format(round(proporcao_valor_internacoes_SP_Brasil,2)))

proporcao_valor_internacoes_SP_sudeste = aux4[aux4['uf']== 'SP']['valor_total'].sum()/aux4[aux4['regiao']== 'sudeste']['valor_total'].sum() *100
print('São Paulo representa {}% do valor total pago em internações na região Sudeste em 2018'.format(round(proporcao_valor_internacoes_SP_sudeste,2)))

In [ ]:
#valor anual pago por habitante

### H2. A maior proporção dos complementos federais em relação ao valor total pago devem é a da região nordeste.
**FALSA.** A maior proporção dos complementos federais em relação ao valor total pago é a da região Sudeste. A regiao nordeste, apesar de ocupar o 2 lugar em valor absoluto de complemento, está em quarta posição na comparação complementos federais x valor total pago

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    plt.subplot(2,1,1)
    aux1 = df4[['regiao', 'complemento_federal']].groupby('regiao').sum().sort_values(by='complemento_federal', ascending=False).reset_index()
    sns.barplot(x='complemento_federal', y='regiao', data=aux1)
    plt.title('Valor total absoluto pago em complemento federal (profissionais + serviços hospitalares) por região')
    plt.xlabel('Valor pago em complemento federal em 10 milhões de R$')
    plt.subplots_adjust(hspace = 0.5)

    plt.subplot(2,1,2)
    aux2 = df4[['regiao', 'complemento_federal', 'valor_total']]
    aux2['proporcao_complemento_federal'] = (aux2['complemento_federal'] / (aux2['complemento_federal'] + aux2['valor_total']) )*100
    aux3 = aux2[['regiao', 'proporcao_complemento_federal']].groupby('regiao').mean().sort_values(by='proporcao_complemento_federal', ascending=False).reset_index()
    sns.barplot(x='proporcao_complemento_federal', y='regiao', data=aux3)
    plt.title('Proporção de complemento federal em comparação com valor total pago por internações por região')
    plt.xlabel('Proporção de complemento federal em comparação com valor total pago por internações por região em %');

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    plt.subplot(1,2,1)
    aux1 = df4[['uf', 'complemento_federal']].groupby('uf').sum().sort_values(by='complemento_federal', ascending=False).reset_index()
    sns.barplot(x='complemento_federal', y='uf', data=aux1)
    plt.title('Valor total absoluto pago em complemento federal (profissionais + serviços hospitalares) por UF')
    plt.xlabel('Valor pago em complemento federal em 10 milhões de R$')
    plt.subplots_adjust(hspace = 0.5)

    plt.subplot(1,2,2)
    aux2 = df4[['uf', 'complemento_federal', 'valor_total']]
    aux2['proporcao_complemento_federal'] = (aux2['complemento_federal'] / (aux2['complemento_federal'] + aux2['valor_total']) )*100
    aux3 = aux2[['uf', 'proporcao_complemento_federal']].groupby('uf').mean().sort_values(by='proporcao_complemento_federal', ascending=False).reset_index()
    sns.barplot(x='proporcao_complemento_federal', y='uf', data=aux3)
    plt.title('Proporção de complemento federal em comparação com valor total pago por internações por UF')
    plt.xlabel('Proporção de complemento federal em comparação com valor total pago por internações por UF em % - média');



In [ ]:
aux = df4[['data', 'valor_total', 'complemento_federal', 'regiao']].groupby(['data', 'regiao']).sum().reset_index()

plt.subplot(1,2,1)
sns.lineplot(x='data', y='complemento_federal', hue='regiao', data=aux)
plt.title('Valores absolutos do valor pago em complemento federal ao longo do tempo')
plt.ylabel('Complemento federal em milhão de R$')

plt.subplot(1,2,2)
sns.lineplot(x='data', y='valor_total', hue='regiao', data=aux)
plt.title('Valores absolutos totais pagos em internação ao longo do tempo')
plt.ylabel('Complemento federal em 100 milhões de R$')

Observações interessantes: pontos de máxima de valor total máximo não coincidem com os de máxima de complemento federal

### H3. A maior proporção dos complementos gestor em relação ao valor total pago deve estar na região sul.
**FALSA.** AA maior proporção dos complementos gestor em relação ao valor total pago está na região centro-oeste. 

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    plt.subplot(2,1,1)
    aux1 = df4[['regiao', 'complemento_gestor']].groupby('regiao').sum().sort_values(by='complemento_gestor', ascending=False).reset_index()
    sns.barplot(x='complemento_gestor', y='regiao', data=aux1)
    plt.title('Valor total absoluto pago em complemento gestor (profissionais + serviços hospitalares) por região')
    plt.xlabel('Valor pago em complemento gestor em 10 milhões de R$')
    plt.subplots_adjust(hspace = 0.5)

    plt.subplot(2,1,2)
    aux2 = df4[['regiao', 'complemento_gestor', 'valor_total']]
    aux2['proporcao_complemento_gestor'] = (aux2['complemento_gestor'] / (aux2['complemento_gestor'] + aux2['valor_total']) )*100
    aux3 = aux2[['regiao', 'proporcao_complemento_gestor']].groupby('regiao').mean().sort_values(by='proporcao_complemento_gestor', ascending=False).reset_index()
    sns.barplot(x='proporcao_complemento_gestor', y='regiao', data=aux3)
    plt.title('Proporção de complemento gestor em comparação com valor total pago por internações por região')
    plt.xlabel('Proporção de complemento gestor em comparação com valor total pago por internações por região em %');

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    plt.subplot(1,2,1)
    aux1 = df4[['uf', 'complemento_gestor']].groupby('uf').sum().sort_values(by='complemento_gestor', ascending=False).reset_index()
    sns.barplot(x='complemento_gestor', y='uf', data=aux1)
    plt.title('Valor total absoluto pago em complemento gestor (profissionais + serviços hospitalares) por UF')
    plt.xlabel('Valor pago em complemento gestor em 10 milhões de R$')
    plt.subplots_adjust(hspace = 0.5)

    plt.subplot(1,2,2)
    aux2 = df4[['uf', 'complemento_gestor', 'valor_total']]
    aux2['proporcao_complemento_gestor'] = (aux2['complemento_gestor'] / (aux2['complemento_gestor'] + aux2['valor_total']) )*100
    aux3 = aux2[['uf', 'proporcao_complemento_gestor']].groupby('uf').mean().sort_values(by='proporcao_complemento_gestor', ascending=False).reset_index()
    sns.barplot(x='proporcao_complemento_gestor', y='uf', data=aux3)
    plt.title('Proporção de complemento gestor em comparação com valor total pago por internações por UF')
    plt.xlabel('Proporção de complemento gestor em comparação com valor total pago por internações por UF em %- média');

In [ ]:
aux = df4[['data', 'valor_total', 'complemento_gestor', 'regiao']].groupby(['data', 'regiao']).sum().reset_index()

plt.subplot(1,2,1)
sns.lineplot(x='data', y='complemento_gestor', hue='regiao', data=aux)
plt.title('Valores absolutos do valor pago em complemento gestor ao longo do tempo')
plt.ylabel('Complemento gestor em milhão de R$')

plt.subplot(1,2,2)
sns.lineplot(x='data', y='valor_total', hue='regiao', data=aux)
plt.title('Valores absolutos totais pagos em internação ao longo do tempo')
plt.ylabel('Complemento gestor em 100 milhões de R$');

### H4. O valor médio por internação dos serviços profissionais deve ser maior na região Sudeste e Sul, respectivamente.
**FALSA.** O valor médio por internação dos serviços profissionais é maior na região Sul e Sudeste, respectivamente. Contudo, o estado com maior valor médio de serviços de profissionais é o Paraná

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    plt.subplot(2,2,1)
    aux = df4[['regiao', 'valor_serviços_profissionais', 'internacoes']]
    aux['valor_serv_prof_internacao'] = aux['valor_serviços_profissionais'] / aux['internacoes']
    aux = aux.groupby('regiao').mean().sort_values(by='valor_serv_prof_internacao', ascending=False).reset_index()
    sns.barplot(x='valor_serv_prof_internacao', y='regiao', data=aux)
    plt.title('Valor médio dos serviços profissionais por região')
    plt.xlabel('Valor médio dos serviçõs profissionais em R$');

    plt.subplot(2,2,(2,4))
    aux = df4[['uf', 'valor_serviços_profissionais', 'internacoes']]
    aux['valor_serv_prof_internacao'] = aux['valor_serviços_profissionais'] / aux['internacoes']
    aux = aux.groupby('uf').mean().sort_values(by='valor_serv_prof_internacao', ascending=False).reset_index()
    sns.barplot(x='valor_serv_prof_internacao', y='uf', data=aux)
    plt.title('Valor médio dos serviços profissionais por UF')
    plt.xlabel('Valor médio dos serviçõs profissionais em R$');

    plt.subplot(2,2,3)
    aux = df4[['data','regiao', 'valor_serviços_profissionais', 'internacoes']]
    aux['valor_serv_prof_internacao'] = aux['valor_serviços_profissionais'] / aux['internacoes']
    aux = aux.groupby(['data', 'regiao']).mean().sort_values(by='valor_serv_prof_internacao', ascending=False).reset_index()
    sns.lineplot(x='data', y='valor_serv_prof_internacao', hue='regiao', data=aux)
    plt.title('Valor médio de serviços profissionais ao longo do tempo por região')
    plt.ylabel('Valor médio de serviços profissionais em R$');

### H5. O maior número médio de leitos ocupados  deve estar na região Sudeste.
**VERDADEIRA.** O maior número médio de leitos ocupados está na região Sudeste

In [ ]:
plt.subplot(2,2,1)
aux = df4[['regiao','uf', 'no_leitos_ocupado']].groupby('regiao').mean().sort_values(by='no_leitos_ocupado', ascending=False).reset_index()
sns.barplot(x='no_leitos_ocupado', y='regiao', data=aux)
plt.title('Média de número de leitos ocupados por região')
plt.xlabel('Média de número de leitos ocupados')

plt.subplot(2,2,(2,4))
aux = df4[['regiao','uf', 'no_leitos_ocupado']].groupby('uf').mean().sort_values(by='no_leitos_ocupado', ascending=False).reset_index()
sns.barplot(x='no_leitos_ocupado', y='uf', data=aux)
plt.title('Média de número de leitos ocupados por UF')
plt.xlabel('Média de número de leitos ocupados')


plt.subplot(2,2,3)
aux = df4[['data', 'regiao', 'no_leitos_ocupado']].groupby(['data', 'regiao']).mean().sort_values(by='no_leitos_ocupado', ascending=False).reset_index()
sns.lineplot(x='data', y='no_leitos_ocupado', hue='regiao', data=aux)
plt.title('Média de número de leitos ocupados por região ao longo do tempo')
plt.ylabel('Média de número de leitos ocupados');

### H6. A região sudeste é a que deve possuir mais leitos ocupados por habitante
**FALSO.** A região Sul é a que possui mais leitos ocupados por habitante, liderados pelo estado do Paraná

In [3]:
plt.subplot(1,2,1)
aux = df4[['no_leitos_ocupado', 'regiao', 'populacao']].groupby('regiao').mean().reset_index()
aux['no_leitos_pop'] = aux['no_leitos_ocupado'] / aux['populacao'] * 100000
aux = aux.sort_values(by='no_leitos_pop', ascending=False)
sns.barplot(x='no_leitos_pop', y='regiao', data=aux)
plt.title('Número médio de leitos ocupados a cada 100 mil habitantes de cada região')
plt.xlabel('Número de leitos ocupados por habitantes /100000 habitantes ');

plt.subplot(1,2,2)
aux = df4[['no_leitos_ocupado', 'uf', 'populacao']].groupby('uf').mean().reset_index()
aux['no_leitos_pop'] = aux['no_leitos_ocupado'] / aux['populacao'] * 100000
aux = aux.sort_values(by='no_leitos_pop', ascending=False)
sns.barplot(x='no_leitos_pop', y='uf', data=aux)
plt.title('Número médio de leitos ocupados a cada 100 mil habitantes de cada UF')
plt.xlabel('Número de leitos ocupados por habitantes /100000 habitantes ');

NameError: name 'plt' is not defined

### H7. Os estados da região Norte devem ser os que possuem menos leitos ocupados por habitantes.
**VERDADEIRA.** Conforme visto em nas plotagens da H6. Porém RR e RO possuem ocupações bem acima da média do restante da região

### H8. O valor médio das internações pagas deve ser maior no estado de São Paulo e Distrito Federal, respectivamente 
**FALSA.** O valor médio das internações pagas é maior no estado do Paraná, Santa Catarina e Rio Grande do Sul, respectivamente. São Paulo e DF ocupam a 7a e 16a posição, respectivamente.

In [ ]:
aux = df4[['valor_medio_intern', 'uf']].groupby('uf').mean().sort_values(by='valor_medio_intern', ascending=False).reset_index()
sns.barplot(y='uf', x='valor_medio_intern', data=aux)
plt.title('Valor médio das internações por UF')
plt.xlabel('Valor médio das internações em R$');

In [ ]:
aux = df4[['data', 'valor_medio_intern', 'uf']].groupby(['data','uf']).mean().sort_values(by='valor_medio_intern', ascending=False).reset_index()
sns.lineplot(x='data', y='valor_medio_intern', hue='uf',data=aux)
plt.title('Valor médio das internações por UF ao longo do tempo')
plt.ylabel('Valor médio das internações em R$');

### H9. O tempo médio de permanência deve ser maior em estados do Nordeste 
**FALSA.** O tempo médio de permanência é maior na região Sudeste, com destaque para o Rio de Janeiro. Para quases todas as regiões há um aumento no tempo médio de permanência em janeiro e julho

In [4]:
plt.subplot(2,2,1)
aux = df4[['regiao', 'media_permanencia']].groupby('regiao').mean().sort_values(by='media_permanencia', ascending=False).reset_index()
sns.barplot(x='media_permanencia', y='regiao', data=aux)
plt.title('Média de permanência em internação por região')
plt.xlabel('Média de permanência em internação em dias')

plt.subplot(2,2,(2,4))
aux = df4[['uf', 'media_permanencia']].groupby('uf').mean().sort_values(by='media_permanencia', ascending=False).reset_index()
sns.barplot(x='media_permanencia', y='uf', data=aux)
plt.title('Média de permanência em internação por UF')
plt.xlabel('Média de permanência em internação em dias');

plt.subplot(2,2,3)
aux = df4[['data', 'regiao', 'media_permanencia']].groupby(['data','regiao']).mean().sort_values(by='media_permanencia', ascending=False).reset_index()
sns.lineplot(x='data', y='media_permanencia', hue='regiao', data=aux)
plt.title('Média de permanência em internação por região ao longo do tempo')
plt.ylabel('Média de permanência em internação em dias');

NameError: name 'plt' is not defined

### H10. A taxa de mortalidade deve ser maior em estados do Norte e Nordesete, respectivamente. 
**FALSA.** A taxa de mortalidade é maior em estados do Sudeste e Sul, respectivamente, com destaque para o Rio de Janeiro. <br>
Em agosto de 2018 a taxa de mortalidade de todas as regiões decaiu, exceto para região sul, que aumentou

In [ ]:
plt.subplot(2,2,1)
aux = df4[['regiao', 'taxa_mortalidade']].groupby('regiao').mean().sort_values(by='taxa_mortalidade', ascending=False).reset_index()
sns.barplot(x='taxa_mortalidade', y='regiao', data=aux)
plt.title('Taxa de mortalidade média por região')
plt.xlabel('Taxa de mortalidade média em %')

plt.subplot(2,2,(2,4))
aux = df4[['uf', 'taxa_mortalidade']].groupby('uf').mean().sort_values(by='taxa_mortalidade', ascending=False).reset_index()
sns.barplot(x='taxa_mortalidade', y='uf', data=aux)
plt.title('Taxa de mortalidade média por UF')
plt.xlabel('Taxa de mortalidade média em %')

plt.subplot(2,2,3)
aux = df4[['data', 'regiao', 'taxa_mortalidade']].groupby(['data','regiao']).mean().sort_values(by='taxa_mortalidade', ascending=False).reset_index()
sns.lineplot(x='data', y='taxa_mortalidade', hue='regiao', data=aux)
plt.title('Taxa de mortalidade média em internação por região ao longo do tempo')
plt.ylabel('Taxa de mortalidade média em %');

### H11. A região Sudeste deve concentrar o maior número de prorrogaões de internação que os outros estados.
**VERDADEIRA.** A região Sudeste concentra o maior número de prorrogações tanto em valor absoltuo quanto em proporção ao total de AIH aprovadas, com destaque para os estados de São Paulo e Rio de Janeiro. Alagoas também possui taxas bastante elevadas de prorrogações

In [5]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    plt.subplot(3,3,1)
    aux=df4[['regiao', 'prorrogacoes']].groupby('regiao').sum().sort_values(by='prorrogacoes', ascending=False).reset_index()
    sns.barplot(x='prorrogacoes', y='regiao', data=aux)
    plt.title('Numero de prorrogacoes absoluto por região')
    plt.xlabel('Numero de prorrogacoes')

    plt.subplot(3,3,(2,5))
    aux=df4[['uf', 'prorrogacoes']].groupby('uf').sum().sort_values(by='prorrogacoes', ascending=False).reset_index()
    sns.barplot(x='prorrogacoes', y='uf', data=aux)
    plt.title('Numero de prorrogacoes absoluto por UF')
    plt.xlabel('Numero de prorrogacoes')
    plt.subplots_adjust(hspace = 0.5)

    plt.subplot(3,3,4)
    aux = df4[['regiao','aih_aprovadas', 'prorrogacoes']]
    aux['proporcao_prorrog'] = (aux['prorrogacoes'] / aux['aih_aprovadas'] *100)
    aux = aux.groupby('regiao').mean().sort_values(by='proporcao_prorrog', ascending=False).reset_index()
    sns.barplot(x='proporcao_prorrog', y='regiao', data=aux)
    plt.title('Proporção de prorrogações em relação as AIH aprovadas por região')
    plt.xlabel('Proporção de prorrogações entre as AIH aprovadas (%)');
    plt.subplots_adjust(hspace = 0.5)

    plt.subplot(3,3,(3,6))
    aux = df4[['uf','aih_aprovadas', 'prorrogacoes']]
    aux['proporcao_prorrog'] = (aux['prorrogacoes'] / aux['aih_aprovadas'] *100)
    aux = aux.groupby('uf').mean().sort_values(by='proporcao_prorrog', ascending=False).reset_index()
    sns.barplot(x='proporcao_prorrog', y='uf', data=aux)
    plt.title('Proporção de prorrogações em relação as AIH aprovadas por UF')
    plt.xlabel('Proporção de prorrogações entre as AIH aprovadas (%)');
    plt.subplots_adjust(hspace = 0.5)

    plt.subplot(3,3,(7,9))
    aux = df4[['data', 'regiao', 'prorrogacoes']].groupby(['data','regiao']).sum().sort_values(by='prorrogacoes', ascending=False).reset_index()
    sns.lineplot(x='data', y='prorrogacoes', hue='regiao', data=aux)
    plt.title('Prorrogações por região ao longo do tempo')
    plt.ylabel('Número de prorrogações');

NameError: name 'plt' is not defined

### H12. O número médio de internações deve ser maior em meses de inverno para todos os estados do Brasil
**FALSO**. O número médio de internações é maior em meses de inverno para quase todos os estados, exceto para o Pará, Maranhão, Piauí e Rondônia. Para os estados em que se aplica, há um aumento médio de 5,52% no número de internações durante os meses de inverno, com destaque para Acre e Distrito Federal

In [ ]:
inverno=['6', '7', '8', '9']
aux = df4[df4['mes'].isin(inverno)]
aux = aux[['internacoes', 'uf']].groupby('uf').mean().sort_values(by='internacoes', ascending=False).reset_index()
aux.columns=['uf', 'internacoes_inverno']

aux1 = df4[~df4['mes'].isin(inverno)]
aux1 = aux1[['internacoes', 'uf']].groupby('uf').mean().sort_values(by='internacoes', ascending=False).reset_index()

aux2 = aux.merge(aux1, on='uf')

aux2['diferenca_%'] = ((aux2['internacoes_inverno'] / aux2['internacoes']) -1)*100

In [6]:
aux2[aux2['diferenca_%'] < 0]

NameError: name 'aux2' is not defined

In [ ]:
aux2[aux2['diferenca_%'] >= 0]['diferenca_%'].mean()

In [7]:
aux2

NameError: name 'aux2' is not defined

### H13. O tempo médio de internação deve ser maior nos meses de inverno
**FALSA.** O tempo médio de internação é menor nos meses de inverno para 10 dos 26 estados. Para os estados em que se aplica, há um aumento de apenas 1,42% no tempo médio de permanência. 

In [ ]:
df4.columns

In [8]:
aux = df4[df4['mes'].isin(inverno)]
aux = aux[['media_permanencia', 'uf']].groupby('uf').mean().sort_values(by='media_permanencia', ascending=False).reset_index()
aux.columns=['uf', 'media_permanencia_inverno']

aux1 = df4[~df4['mes'].isin(inverno)]
aux1 = aux1[['media_permanencia', 'uf']].groupby('uf').mean().sort_values(by='media_permanencia', ascending=False).reset_index()

aux2 = aux.merge(aux1, on='uf')

aux2['diferenca_%'] = ((aux2['media_permanencia_inverno'] / aux2['media_permanencia']) -1)*100

NameError: name 'df4' is not defined

In [ ]:
aux2[aux2['diferenca_%'] < 0]

In [9]:
aux2[aux2['diferenca_%'] >= 0]['diferenca_%'].mean()

NameError: name 'aux2' is not defined

In [ ]:
aux2

### H14. O comportamento das internações no tempo deve ser igual para todos os estados do Brasil
**VERDADEIRA.** O comportamento das internações no tempo (crescimentos e declínios é igual para todos os estados do Brasil

In [10]:
plt.title('Evolução do número de internações por UF e região')

plt.subplot(5,1,1)
aux = df4[['data', 'regiao', 'uf', 'internacoes']].groupby(['data','regiao', 'uf']).sum().reset_index()
aux = aux[aux['regiao'] == 'sul']
sns.lineplot(x='data', y='internacoes', hue='uf', data=aux)


plt.subplot(5,1,2)
aux = df4[['data', 'regiao', 'uf', 'internacoes']].groupby(['data','regiao', 'uf']).sum().reset_index()
aux = aux[aux['regiao'] == 'sudeste']
sns.lineplot(x='data', y='internacoes', hue='uf', data=aux)

plt.subplot(5,1,3)
aux = df4[['data', 'regiao', 'uf', 'internacoes']].groupby(['data','regiao', 'uf']).sum().reset_index()
aux = aux[aux['regiao'] == 'centro-oeste']
sns.lineplot(x='data', y='internacoes', hue='uf', data=aux)

plt.subplot(5,1,4)
aux = df4[['data', 'regiao', 'uf', 'internacoes']].groupby(['data','regiao', 'uf']).sum().reset_index()
aux = aux[aux['regiao'] == 'nordeste']
sns.lineplot(x='data', y='internacoes', hue='uf', data=aux)

plt.subplot(5,1,5)
aux = df4[['data', 'regiao', 'uf', 'internacoes']].groupby(['data','regiao', 'uf']).sum().reset_index()
aux = aux[aux['regiao'] == 'norte']
sns.lineplot(x='data', y='internacoes', hue='uf', data=aux);

NameError: name 'plt' is not defined

## RESUMO DOS PRINCIPAIS INSIGHTS

- A região Sudeste, por ser a mais populosa do país, concentra o maior número absoluto de internações e, consequentemente, com gastos para pagamentos das internações (mais de 50% do valor total gasto em internações do Brasil são de São Paulo);
- A região Sudeste é a que mais tem repasses do governo global para complemento de pagamentos de serviços hospitalares e profissionais da saúde. Porém Goiás, Bahia e Rio de Janeiro são os que mais recebem complementos federais em relação aos valores totais das internações. Interessante perceber que os meses cuja média de complementos federais são maiores não correspondem aos meses em que os valores totais são maiores;
- A região centro-oeste é a que tem mais repasses do gestor (estadual ou municipal) comparado com o valor total, com destaque novamente para o Goiás;
- Em dezembro de 2017 a região nordeste teve um grande repasse dos gestores;
- A região Sul é a que tem o maior valor médio gasto por internação, com destaque para o Paraná;
- A região Sul é a que tem mais leitos ocupados por 100 mil habitantes. Essa informação deve ser comparada com a disponibilidade de leitos para melhor planejamento estratégico; 
- O tempo médio de permanência em internação é maior na região sudeste, com destaque para o Rio de Janeiro;
- A taxa de mortalidade é maior na região Sudeste, com destaque ṕara o Rio de Janeiro;
- São Paulo, Rio de Janeiro e Alagoas apresentam elevada taxa de prorrogração de internações;
- Durante o inverno, há um aumento no número médio de internações para alguns estados, assim como no tempo médio de internação;
- Todos os estados tem comportamento semelhante do número de internações no tempo

<p><b>Conclusão:</b> Além do número de internações, é preciso observar outros aspectos da gestão. Algumas perguntas para investigações mais profundas podem ser feitas, como por exemplo:
    
    - Por que no Rio de Janeiro tanto o tempo de internação quanto o número de prorrogações e taxa de mortalidade são mais altos? Os casos são mais complexos? Há equipamentos suficientes para atender a população?
    - Por que a região sul tem um valor médio pago por internação maiores? Complexidade dos procedimentos?
    - Por que em agosto de 2018 a taxa de mortalidade média caiu em todas as regiões, exceto na região sul?
    - Quais as possíveis causas da média do número de internações aumentar nos meses de inverno? 
    
    


# 5.0 Preparação dos dados

In [ ]:
df5 = df4.copy()

In [ ]:
#duas maneiras (existem várias) -- média móvel, autoregressão, trazer o próximo valor, média do dataset

## 5.1 Imputação de dados com a média móvel

In [11]:
df5.head()

NameError: name 'df5' is not defined

In [ ]:
df_media_movel = pd.DataFrame(columns=df5.columns)
uf = list(df5['uf'].unique())

for i in range(0,len(uf)):
    df_uf = df5.loc[df5['uf'] == uf[i]]
    df_uf = df_uf.fillna(df_uf[['internacoes', 'valor_total', 'valor_medio_aih', 'obitos']].rolling(6, min_periods=0).mean())
    df_media_movel = pd.concat([df_media_movel,df_uf])
#df_teste = df5.loc[df5['uf'] == 'SP']

In [12]:
sns.lineplot(x='data', y='internacoes', data=df_media_movel.groupby('data').sum().reset_index())

NameError: name 'sns' is not defined

# 6.0 Modelagem dos dados

In [13]:
df6 = df_media_movel[['data', 'internacoes', 'obitos']].groupby('data').sum().reset_index()
df6.head()

NameError: name 'df_media_movel' is not defined

## 6.1 Internações

In [ ]:
df6_internacoes = df6[['data','internacoes']]

df6_internacoes.index = df6_internacoes['data']
df6_internacoes = df6_internacoes.drop(columns=['data'])

df6_internacoes.head()

#teste de estacionariedade
dftest = adfuller(df6_internacoes, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Teste', 'Valor p', 'No de lags', 'No de observações'])
dfoutput



stepwise_model = auto_arima(df6_internacoes, start_p=1, start_q=1, max_q=6, m=6, seasonal=True, d=1, D=1, trace=True, error_action='ignore',
                            suppress_warnings=True, stepwise=True)

print(stepwise_model.aic())

train = df6_internacoes.loc['2017-12-01':'2019-02-01']
test = df6_internacoes.loc['2019-02-01':]

stepwise_model.fit(train)

future_forecast = stepwise_model.predict(n_periods=6)


future_forecast = pd.DataFrame(future_forecast,index = test.index,columns=['internacoes'])
df6_internacoes_prev = pd.concat([test,future_forecast],axis=1)

df6_internacoes_prev.columns = ['real', 'previsto']
df6_internacoes_prev['erro absoluto'] = df6_internacoes_prev['previsto'] - df6_internacoes_prev['real'] 
df6_internacoes_prev['erro percentual'] = ( df6_internacoes_prev['previsto'] - df6_internacoes_prev['real'] ) / df6_internacoes_prev['real'] *100
MAPE = df6_internacoes_prev['erro percentual'].mean()
print(MAPE)

df6_internacoes_prev

## 6.2 Óbitos

In [14]:
df6_obitos = df6[['data','obitos']]

df6_obitos.index = df6_obitos['data']
df6_obitos = df6_obitos.drop(columns=['data'])

df6_obitos.head()

#teste de estacionariedade
dftest = adfuller(df6_obitos, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Teste', 'Valor p', 'No de lags', 'No de observações'])
dfoutput



stepwise_model = auto_arima(df6_obitos, start_p=1, start_q=1, max_q=6, m=6, seasonal=True, d=1, D=1, trace=True, error_action='ignore',
                            suppress_warnings=True, stepwise=True)

print(stepwise_model.aic())

train = df6_obitos.loc['2017-12-01':'2019-02-01']
test = df6_obitos.loc['2019-02-01':]

stepwise_model.fit(train)

future_forecast = stepwise_model.predict(n_periods=6)


future_forecast = pd.DataFrame(future_forecast,index = test.index,columns=['obitos'])
df6_obitos_prev = pd.concat([test,future_forecast],axis=1)

df6_obitos_prev.columns = ['real', 'previsto']
df6_obitos_prev['erro absoluto'] = df6_obitos_prev['previsto'] - df6_obitos_prev['real'] 
df6_obitos_prev['erro percentual'] = ( df6_obitos_prev['previsto'] - df6_obitos_prev['real'] ) / df6_obitos_prev['real'] *100
MAPE = df6_obitos_prev['erro percentual'].mean()
print(MAPE)

df6_obitos_prev

NameError: name 'df6' is not defined

## 6.3 Valor médio por AIH

In [ ]:
df6_valor_medio_aih = df_media_movel[['data', 'valor_medio_aih']].groupby('data').mean()

df6_valor_medio_aih.head()

#teste de estacionariedade
dftest = adfuller(df6_valor_medio_aih, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Teste', 'Valor p', 'No de lags', 'No de observações'])
dfoutput



stepwise_model = auto_arima(df6_valor_medio_aih, start_p=1, start_q=1, max_q=6, m=6, seasonal=True, d=1, D=1, trace=True, error_action='ignore',
                            suppress_warnings=True, stepwise=True)

print(stepwise_model.aic())

train = df6_valor_medio_aih.loc['2017-12-01':'2019-02-01']
test = df6_valor_medio_aih.loc['2019-02-01':]

stepwise_model.fit(train)

future_forecast = stepwise_model.predict(n_periods=6)


future_forecast = pd.DataFrame(future_forecast,index = test.index,columns=['valor_medio_aih'])
df6_valor_medio_aih_prev = pd.concat([test,future_forecast],axis=1)

df6_valor_medio_aih_prev.columns = ['real', 'previsto']
df6_valor_medio_aih_prev['erro absoluto'] = df6_valor_medio_aih_prev['previsto'] - df6_valor_medio_aih_prev['real'] 
df6_valor_medio_aih_prev['erro percentual'] = ( df6_valor_medio_aih_prev['previsto'] - df6_valor_medio_aih_prev['real'] ) / df6_valor_medio_aih_prev['real'] *100
MAPE = df6_valor_medio_aih_prev['erro percentual'].mean()
print(MAPE)

df6_valor_medio_aih_prev

# 7.0 Previsões e avaliações finais

In [ ]:
df7 = df_media_movel[['data', 'internacoes', 'obitos']].groupby('data').sum().reset_index()
df7.head()

## 6.1 Internações

In [15]:
df7_internacoes = df7[['data','internacoes']]

df7_internacoes.index = df7_internacoes['data']
df7_internacoes = df7_internacoes.drop(columns=['data'])

df7_internacoes.head()

#teste de estacionariedade
dftest = adfuller(df7_internacoes, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Teste', 'Valor p', 'No de lags', 'No de observações'])
dfoutput



stepwise_model = auto_arima(df7_internacoes, start_p=1, start_q=1, max_q=6, m=6, seasonal=True, d=1, D=1, trace=True, error_action='ignore',
                            suppress_warnings=True, stepwise=True)

print(stepwise_model.aic())

stepwise_model.fit(df7_internacoes)

future_forecast = stepwise_model.predict(n_periods=6)


future_forecast_int = pd.DataFrame(future_forecast, index=data_prev, columns=['previsao_internacoes'])

future_forecast_int

NameError: name 'df7' is not defined

In [ ]:
data_prev = ['2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01']

## 6.2 Óbitos

In [16]:
df7_obitos = df7[['data','obitos']]

df7_obitos.index = df7_obitos['data']
df7_obitos = df7_obitos.drop(columns=['data'])

df7_obitos.head()

#teste de estacionariedade
dftest = adfuller(df7_obitos, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Teste', 'Valor p', 'No de lags', 'No de observações'])
dfoutput



stepwise_model = auto_arima(df7_obitos, start_p=1, start_q=1, max_q=6, m=6, seasonal=True, d=1, D=1, trace=True, error_action='ignore',
                            suppress_warnings=True, stepwise=True)

print(stepwise_model.aic())

stepwise_model.fit(df7_obitos)

future_forecast = stepwise_model.predict(n_periods=6)


future_forecast_obito = pd.DataFrame(future_forecast, index=data_prev, columns=['previsao_obitos'])

future_forecast_obito

NameError: name 'df7' is not defined

## 6.3 Valor médio por AIH

In [ ]:
df7_valor_medio_aih = df_media_movel[['data', 'valor_medio_aih']].groupby('data').mean()

df7_valor_medio_aih.head()

#teste de estacionariedade
dftest = adfuller(df7_valor_medio_aih, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Teste', 'Valor p', 'No de lags', 'No de observações'])
dfoutput



stepwise_model = auto_arima(df7_valor_medio_aih, start_p=1, start_q=1, max_q=6, m=6, seasonal=True, d=1, D=1, trace=True, error_action='ignore',
                            suppress_warnings=True, stepwise=True)

print(stepwise_model.aic())

stepwise_model.fit(df7_valor_medio_aih)

future_forecast = stepwise_model.predict(n_periods=6)


future_forecast_aih = pd.DataFrame(future_forecast, index=data_prev, columns=['previsao_valor_medio_aih'])

future_forecast_aih

In [17]:
final_predict = pd.concat([future_forecast_int, future_forecast_aih], axis=1)
final_predict['orcamento_bilhao_R$'] = final_predict['previsao_internacoes'] * final_predict['previsao_valor_medio_aih'] / 10**9
final_predict

NameError: name 'future_forecast_int' is not defined

In [ ]:
1.000.000.000